In [2]:
# Let's collect some tools for this job...
import tensorflow.keras as keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import tensorflow as tf
import csv
import os
import replay
%matplotlib inline

In [ ]:
# Use this codeblock if you are using a GPU and have 6gb of VRAM or less
# You might be fine without it, but this just makes it so tensorflow doesn't
# absorb all of it at once.
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# for debugging, train for a specified time
import time
class TimeOut(keras.callbacks.Callback):
    def __init__(self, t0, timeout):
        super().__init__()
        self.t0 = t0
        self.timeout = timeout  # time in minutes

    def on_train_batch_end(self, batch, logs=None):
        if time.time() - self.t0 > self.timeout * 60:  # 58 minutes
            print(f"\nReached {(time.time() - self.t0) / 60:.3f} minutes of training, stopping")
            self.model.stop_training = True

callback = [TimeOut(t0=time.time(), timeout=30)]

#### This reads in every replay found in scrape-out/

#### It achieves this by checking every replay found in scrape-done, which has also been cleaned of user data.

#### You might have to download the used data from an external source.

In [ ]:
# first we need to get every replay that has been successfully parsed
dataX = []
dataY = []
for path, dirs, files in os.walk("scrape-done/"):
    # then we need to load in every file into memory
    for filename in files:
        csvName = "scrape-out/" + filename[:-7] # removes '.json'
        theInput = csvName + '-in.csv'
        passFlag = False
        with open(theInput) as fp:
            reader = csv.reader(fp)
            skip = True
            curReplay = []
            for row in reader:
                if skip:
                    skip = False
                    continue
                rowFixed = row
                # in the event an errored replay exists
                # usually this means a bug in the parser
                if (len(row) != 207):
                    print("len not 207:", len(row))
                    print(theInput)
                    passFlag = True
                    break
                # convert all values in a row to float32
                for x in range(len(row)):
                    rowFixed[x] = np.float32(row[x])
                curReplay.append(np.array(rowFixed))
            # 269 turns is our longest replay in the set
            for x in range(269 - len(curReplay)):
                curReplay.append(np.full((207),np.float32(-1.0)))
        dataX.append(curReplay)
        theOutput = csvName + "-out.csv"
        if passFlag:
            passFlag = False
            continue
        with open(theOutput) as fp:
            reader = csv.reader(fp)
            skip = True
            curReplay2 = []
            for row in reader:
                if skip:
                    skip = False
                    continue
                rowFixed = row[0:2]
                moveRow = np.full((6), np.float32(-1.0))
                moveRow[int(rowFixed[0])] += 2 
                switchRow = np.full((6), np.float32(-1.0))
                if (int(rowFixed[1]) > 0):
                    switchRow[int(rowFixed[1])] += 2
                curReplay2.append(np.array([moveRow, switchRow]).flatten())
            for x in range(269 - len(curReplay2)):
                curReplay2.append(np.full((12), np.float32(-1.0)))
        dataY.append(curReplay2)

#### Printing some stuff to look at

In [ ]:
print(len(dataX))
print(len(dataX[0]))
print(len(dataX[0][0]))

In [ ]:
dataX[0][0]

In [ ]:
print(len(dataY))
print(len(dataY[0]))

In [ ]:
dataY[0][0:4]

#### This slices arrays. You can set the 1000 to something higher if you have more RAM.

In [ ]:
X = np.array(dataX)
# the "current" turn
splice = X.shape[0]
preX = X[0:1000,0:-1]
# the move taken in the current turn
Y = np.array(dataY)
preY = Y[0:1000,0:-1]

In [ ]:
print(preX.shape)
print(preY.shape)

#### Setup the layers

In [ ]:
hidden_size = 512
input_size = X.shape[2]

model = keras.models.Sequential()
# input_layer = layers.Input(shape=(None, input_size))
model.add(layers.LSTM(hidden_size, 
                      input_shape=(preX.shape[1], input_size), 
                      return_sequences=True))
model.add(layers.Dense(12, input_shape=(None, input_size)))

# Compile it...
model.compile(loss=keras.losses.CategoricalCrossentropy(),
 optimizer=keras.optimizers.Adam(),
 metrics=[keras.metrics.CategoricalAccuracy()])
model.summary()

#### Show the layers

In [ ]:
keras.utils.plot_model(model,
 show_shapes=True,expand_nested=True)

#### Train

In [ ]:
start = time.time()
batch_size = preX.shape[0] // 2 # number of patterns...
epochs = 50
history = model.fit(preX, preY,
    batch_size=batch_size,
    epochs=epochs,
    callbacks = callback,
    verbose=0)
print('Accuracy:',model.evaluate(preX, preY)[1]*100.0,'%')
print("time to calculate:", time.time() - start)

In [ ]:
plt.figure(1)
# summarize history for accuracy
plt.subplot(211)
plt.plot(history.history['categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
# summarize history for loss
plt.subplot(212)
plt.plot(history.history['loss'][1:])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.tight_layout()
plt.show()

#### From here, we can test

To start, you need the team you are going to play with in team.txt, and the opponents pokemon (as they appear in the js development console) in opp.txt.

Then you need to pick the lead, and have all the text that appears in the console that starts with a "|" at the beginning, including the |turn|1 part

After that, run through the first turn blocks until the next description block

In [ ]:
# imports your own team
team = []
poke_str = [[], []]
with open("team.txt", "r") as fp:
    for x in range(6):
        poke = replay.Pokemon()
        poke.ImportPS(fp)
        poke_str[0].append(poke.Pkmn.name.capitalize())
        team.append(poke)

In [ ]:
# imports the opponents team based on text you gathered
opponentTeam = []
temp = replay.Replay(None, None, True)
with open("opp.txt", "r") as fp:
    enemyParty, pokeStr = temp.GetTeam(fp)
pokemon = [team, enemyParty]
poke_str[1] = pokeStr

In [ ]:
poke_str

In [ ]:
# import turns as you need to
def ManualTurn(pokemon, poke_str, prev_turn):
    with open("temp.txt", "w") as fpw:
        print("Enter lines that appear in the javascript console. (ctrl+shift+I in chrome)")
        print("When you have added all text for the turn, include |turn|x where x is the next turn number")
        print("If this is team preview, just put |start in and the two leads")
        user = input("")
        while (user != ""):
            fpw.writelines(user + "\n")
            user = input("")
    with open("temp.txt", "r") as fp:
        turn = replay.Turn(fp, pokemon, poke_str, prev_turn)
            
    return turn
def AutoTurn(pokemon, poke_str, prev_turn):
    with open("temp.txt", "r") as fp:
        turn = replay.Turn(fp, pokemon, poke_str, prev_turn)
    return turn
    
def getAnswer(turns, model):
    turnVector = []
    for x in turns:
        inVec, outVec = x.GetVector()
        turnVector.append(inVec)
    for x in range(268 - len(turnVector)):
        turnVector.append(np.full((207), np.float32(-1.0)))

    vec = np.array([turnVector])
    test = model.predict(vec)
    return test

def printArray(arr, start, end):
    i = 0
    for x in arr[0][start:end]:
        print("Turn:", i)
        for y in x[0:6]:
            print(" {:.2f}".format(y), end=', ')
        print()
        for y in x[6:]:
            print(" {:.2f}".format(y), end=', ')
        print()
        i += 1

In [ ]:
firstTurn = AutoTurn(pokemon, poke_str, None)

The following block will output 12 numbers. The first 4 are your moves and how "good" they are. The 5th number is meaningless, and the 6th is to switch. Pick the highest number in this list.

If you need to switch, the next 6 numbers are how "good" a switch option will be. Again, pick the highest in the list that is alive if you need to switch. 

In [ ]:
turns = [firstTurn]
test = getAnswer(turns, model)
print(test.shape)
printArray(test, 0, 1)

At this point, you should be at a screen where you pick your move in game. At this point, you need to paste all text that starts with a "|" until |turn|x. Exclude the large block that contains your entire team, sometimes the blocks will be separated if you needed to pick a Pokemon to switch to. Continue to do the steps explained. 

You can keep running the below two blocks until the battle ends, and it should keep track of the previous turns magically (I really don't know why this is or if it is persistent but in my working demo, the previous turns never changed).

In [ ]:
turns.append(AutoTurn(pokemon, poke_str, turns[len(turns) - 1]))

In [ ]:
test = getAnswer(turns, model)
printArray(test, 0, len(turns))